In [ ]:
import copy
from operator import attrgetter
import time


class tiles:
    def __init__(self, lst, parent, cost, mov):
        self.lst = lst
        self.parent = parent
        self.cost = cost
        self.mov = mov

    def parent_cost(self):
        count = 0
        while self.parent != [[]]:
            count = count + 1
            self = self.parent
        return count


def free(lst):
    for i in range(0, 3):
        for j in range(0, 3):
            if lst[i][j] == -1:
                return i, j


def movleft(lst, row, col):
    newlst = copy.deepcopy(lst)
    newlst[row][col], newlst[row][col -
                                  1] = newlst[row][col - 1], newlst[row][col]
    return newlst


def movright(lst, row, col):
    newlst = copy.deepcopy(lst)
    newlst[row][col], newlst[row][col +
                                  1] = newlst[row][col + 1], newlst[row][col]
    return newlst


def movdown(lst, row, col):
    newlst = copy.deepcopy(lst)
    newlst[row + 1][col], newlst[row][col] = newlst[row][col], newlst[row + 1][col]
    return newlst


def movup(lst, row, col):
    newlst = copy.deepcopy(lst)
    newlst[row - 1][col], newlst[row][col] = newlst[row][col], newlst[row - 1][col]
    return newlst


def retindex(lst1, num):
    for i in range(0, 3):
        for j in range(0, 3):
            if lst1[i][j] == num:
                return i, j


def misplacedtiles(lst1, lst2):
    count = 0
    for i in range(0, 3):
        for j in range(0, 3):
            if lst1[i][j] != lst2[i][j] and lst1[i][j] != -1:
                count = count + 1
    return count


def manhattandistance(lst1, lst2):
    final = 0
    for i in range(0, 3):
        for j in range(0, 3):
            if lst1[i][j] != -1:
                result = retindex(lst2, lst1[i][j])
                final += abs(i - result[0]) + abs(j - result[1])
    return final


def ucs(start, goal):
    queue = []
    visited = []
    genisis_parent = tiles([[]], [[]], 0, "NULL")
    st = tiles(start, genisis_parent, 0, "No move")
    queue.append(st)
    current = queue.pop(0)
    i = 0
    cost = 1

    while current.lst != goal:
        freetile = free(current.lst)
        visited.append(current.lst)

        i += 1

        if freetile[0] - 1 >= 0:
            tileup = tiles(
                movup(current.lst, freetile[0], freetile[1]), current, cost, "UP")
            if tileup.lst not in visited:
                queue.append(tileup)

        if freetile[0] + 1 <= 2:
            tiledown = tiles(
                movdown(current.lst, freetile[0], freetile[1]), current, cost, "DOWN")
            if tiledown.lst not in visited:
                queue.append(tiledown)

        if freetile[1] - 1 >= 0:
            tileleft = tiles(
                movleft(current.lst, freetile[0], freetile[1]), current, cost, "LEFT")
            if tileleft.lst not in visited:
                queue.append(tileleft)

        if freetile[1] + 1 <= 2:
            tilergt = tiles(
                movright(current.lst, freetile[0], freetile[1]), current, cost, "RIGHT")
            if tilergt.lst not in visited:
                queue.append(tilergt)

        queue.sort(key=attrgetter("cost"))

        if current.parent != queue[0].parent:
            # print("PARENT CHANGED")
            cost += 1

        current = queue.pop(0)

        print("Iteration", i)
        print("parent = ", current.parent.lst)
        print(current.lst, "cost : ", current.cost, "  move on parent : ", current.mov)

        if current.lst == goal:
            print("optimal path : ")
            temp = current
            arr = []
            arr.append(current.lst)
            while temp.parent != [[]]:
                arr.append(temp.parent.lst)
                temp = temp.parent

            for ele in arr:
                if ele == [[]]:
                    arr.remove(ele)
            ans = arr[::-1]
            iter = 0
            for ele in ans:
                print("Level : ", iter)
                print(ele)
                iter += 1
            print(len(queue))
            print(len(visited))
            return i


def gbfs(start, goal, basis):
    queue = []
    visited = []
    genisis_parent = tiles([[]], [[]], 0, "NULL")
    st = tiles(start, genisis_parent, 0, "NO")
    queue.append(st)
    current = queue.pop(0)
    i = 0

    while current.lst != goal:
        freetile = free(current.lst)
        visited.append(current.lst)
        if basis == "MT":
            current.cost = misplacedtiles(current.lst, goal)
        elif basis == "MD":
            current.cost = manhattandistance(current.lst, goal)
        i += 1

        if freetile[0] - 1 >= 0:

            tmpup = movup(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                ucost = misplacedtiles(tmpup, goal)
            elif basis == "MD":
                ucost = manhattandistance(tmpup, goal)

            tileup = tiles(tmpup, current, ucost, "UP")
            if tileup.lst not in visited:
                queue.append(tileup)

        if freetile[0] + 1 <= 2:
            tmpdown = movdown(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                dcost = misplacedtiles(tmpdown, goal)
            elif basis == "MD":
                dcost = manhattandistance(tmpdown, goal)
            tiledown = tiles(tmpdown, current, dcost, "DOWN")
            if tiledown.lst not in visited:
                queue.append(tiledown)

        if freetile[1] - 1 >= 0:
            tmpleft = movleft(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                lcost = misplacedtiles(tmpleft, goal)
            elif basis == "MD":
                lcost = manhattandistance(tmpleft, goal)
            tileleft = tiles(tmpleft, current, lcost, "LEFT")
            if tileleft.lst not in visited:
                queue.append(tileleft)

        if freetile[1] + 1 <= 2:
            tmprgt = movright(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                rcost = misplacedtiles(tmprgt, goal)
            elif basis == "MD":
                rcost = manhattandistance(tmprgt, goal)
            tilergt = tiles(tmprgt, current, rcost, "RIGHT")
            if tilergt.lst not in visited:
                queue.append(tilergt)

        queue.sort(key=attrgetter("cost"))
        """""
        if current.parent != queue[0].parent:
            # print("PARENT CHANGED")
            cost += 1
        """
        current = queue.pop(0)

        print("Iteration", i)
        print("parent = ", current.parent.lst, " cost = ", current.parent.cost)
        print("current = ", current.lst, "cost = ", current.cost, "move on parent : ", current.mov)

        if current.lst == goal:
            print("optimal path : ")
            temp = current
            arr = []
            arr.append(current.lst)
            while temp.parent != [[]]:
                arr.append(temp.parent.lst)
                temp = temp.parent
            for ele in arr:
                if ele == [[]]:
                    arr.remove(ele)
            ans = arr[::-1]
            iter = 0
            for ele in ans:
                print("Level : ", iter)
                print(ele)
                iter += 1
            print(len(queue))
            print(len(visited))
            return i


def a_star(start, goal, basis):
    queue = []
    visited = []
    genisis_parent = tiles([[]], [[]], 0, "NULL")
    st = tiles(start, genisis_parent, 0, "NO")
    queue.append(st)
    current = queue.pop(0)
    i = 0
    # cost = 1

    while current.lst != goal:
        freetile = free(current.lst)
        visited.append(current.lst)
        if basis == "MT":
            current.cost = misplacedtiles(current.lst, goal)
        elif basis == "MD":
            current.cost = manhattandistance(current.lst, goal)
        i += 1

        if freetile[0] - 1 >= 0:

            tmpup = movup(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                ucost = misplacedtiles(tmpup, goal)
            elif basis == "MD":
                ucost = manhattandistance(tmpup, goal)
            val1 = current.parent_cost()
            realtileup = tiles(tmpup, current, ucost + val1 + 1, "UP")
            if realtileup.lst not in visited:
                queue.append(realtileup)

        if freetile[0] + 1 <= 2:
            tmpdown = movdown(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                dcost = misplacedtiles(tmpdown, goal)
            elif basis == "MD":
                dcost = manhattandistance(tmpdown, goal)
            val2 = current.parent_cost()
            realtiledown = tiles(tmpdown, current, dcost + val2 + 1, "DOWN")
            if realtiledown.lst not in visited:
                queue.append(realtiledown)

        if freetile[1] - 1 >= 0:
            tmpleft = movleft(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                lcost = misplacedtiles(tmpleft, goal)
            elif basis == "MD":
                lcost = manhattandistance(tmpleft, goal)
            val3 = current.parent_cost()
            realtileleft = tiles(tmpleft, current, lcost + val3 + 1, "LEFT")
            if realtileleft.lst not in visited:
                queue.append(realtileleft)

        if freetile[1] + 1 <= 2:
            tmprgt = movright(current.lst, freetile[0], freetile[1])
            if basis == "MT":
                rcost = misplacedtiles(tmprgt, goal)
            elif basis == "MD":
                rcost = manhattandistance(tmprgt, goal)
            val4 = current.parent_cost()
            realtilergt = tiles(tmprgt, current, rcost + val4 + 1, "RIGHT")
            if realtilergt.lst not in visited:
                queue.append(realtilergt)

        queue.sort(key=attrgetter("cost"))
        """""
        if current.parent != queue[0].parent:
            # print("PARENT CHANGED")
            cost += 1
        """
        current = queue.pop(0)

        print("Iteration", i)
        print("parent = ", current.parent.lst, " cost = ", current.parent.cost)
        print("current = ", current.lst, "cost = ", current.cost, "move on parent : ", current.mov)

        if current.lst == goal:
            print("path : ")
            temp = current
            arr = []
            moves = []
            arr.append(current.lst)
            while temp.parent != [[]]:
                arr.append(temp.parent.lst)
                temp = temp.parent

            for ele in arr:
                if ele == [[]]:
                    arr.remove(ele)
            ans = arr[::-1]
            iter = 0
            for ele in ans:
                print("Level : ", iter)
                print(ele)
                iter += 1
            print(len(queue))
            print(len(visited))
            return i

def stateinput(lst):
    x = 0
    for i in range(0,3):
        for j in range(0,3):
            x = int(input(f"enter position [{i}][{j}] : "))
            lst.append(x)
    lst1 = lst[0:3]
    lst2 = lst[3:6]
    lst3 = lst[6:9]
    lst.clear()
    lst.append(lst1)
    lst.append(lst2)
    lst.append(lst3)
    return lst





initial = [
    [1, 3, 6],
    [-1, 4, 7],
    [2, 5, 8],
]
goal = [
    [1, 3, 6],
    [7, 4, -1],
    [2, 5, 8],
]
"""""
initial = []
stateinput(initial)
print(initial)
final = []
stateinput(final)
print(final)
"""

"""""
print("iterations needed : ", ucs(initial, goal))

base = input("enter basis for gbfs\nit can either be misplaced tiles use MT for it\n"
           "or it can be manhattan distance use MD for it\nOtherwise error\nEnter parameters : ")
if base == "MT":
    print("iterations needed : ", gbfs(initial, goal, "MT"))
elif base == "MD":
    print("iterations needed : ", gbfs(initial, goal, "MD"))
else:
    print("invalid parameters")
"""
base = input("enter basis for astar search\nit can either be misplaced tiles use MT for it\n"
           "or it can be manhattan distance use MD for it\nOtherwise error\nEnter parameters : ")
if base == "MT":
    print("iterations needed : ", a_star(initial, goal, "MT"))
elif base == "MD":
    print("iterations needed : ", a_star(initial, goal, "MD"))
else:
    print("invalid parameters")
